In [2]:
from tensorflow.keras.models import load_model

2024-01-26 12:57:51.701349: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-26 12:57:51.705279: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-26 12:57:51.785328: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-26 12:57:51.790879: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-26 12:57:53.120550: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

Load the models from the previously saved as .h5 files

In [3]:
CNN_31 = load_model('Models/FinalModel_31.h5')
CNN_128 = load_model('Models/FinalModel_128.h5')

In [4]:
CNN_31.summary()

Model: "FinalModel"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 data_rescaling (Sequential  (None, None, None, 1)     0         
 )                                                               
                                                                 
 conv2d_78 (Conv2D)          (None, 31, 31, 64)        1664      
                                                                 
 max_pooling2d_70 (MaxPooli  (None, 15, 15, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_79 (Conv2D)          (None, 15, 15, 128)       73856     
                                                                 
 max_pooling2d_71 (MaxPooli  (None, 7, 7, 128)         0         
 ng2D)                                                           
                                                        

In [5]:
CNN_128.summary()

Model: "BalancedModel"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 data_rescaling (Sequential  (None, None, None, 1)     0         
 )                                                               
                                                                 
 conv2d_11 (Conv2D)          (None, 64, 64, 32)        1600      
                                                                 
 max_pooling2d_10 (MaxPooli  (None, 32, 32, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_12 (Conv2D)          (None, 16, 16, 64)        51264     
                                                                 
 max_pooling2d_11 (MaxPooli  (None, 8, 8, 64)          0         
 ng2D)                                                           
                                                     

In [9]:
# Save the models but this time as tf for tensorflow serving
version = 1
file_path_31 = f"./img_classifier/31/{version}"
file_path_128 = f"./img_classifier/128/{version}"

CNN_31.save(filepath=file_path_31, save_format='tf')
CNN_128.save(filepath=file_path_128, save_format='tf')

INFO:tensorflow:Assets written to: ./img_classifier/31/1/assets


INFO:tensorflow:Assets written to: ./img_classifier/31/1/assets


INFO:tensorflow:Assets written to: ./img_classifier/128/1/assets


INFO:tensorflow:Assets written to: ./img_classifier/128/1/assets


### Deploying Model Locally

**Step 1**: First, head over to the img_classifier directory and create the file 'models.config'

``` bash
~: cd DLModel/img_classifier
~DLModel/img_classifier: mkdir local_config
~DLModel/img_classifier: code models.config
```

**Step 2**: Inside `models.config` we will add the following:
```json
model_config_list: { 
  config: {
    name: "31x31",
    base_path: "/models/img_classifier/31",
    model_platform: "tensorflow"
  },
  config: {
    name: "128x128",
    base_path: "/models/img_classifier/128",
    model_platform: "tensorflow"
  },
}
```


**Step 3**: Finally, we can create the docker container using tensorflow serving
```docker
docker run --name vegetable_server -p 8501:8501 -v "C:/Users/ryany/OneDrive/Documents/SP Y2S2/DevOps/CA2/ca2-daaa2b01-2214452-ryanyeo-dlmodelapp-main/DLModel:/models" -t tensorflow/serving --model_config_file=/models/img_classifier/local_config/models.config &
```

**Step 4**: To test whether the server is running correctly, we would just have to check two urls:
- http://localhost:8501/v1/models/31x31
- http://localhost:8501/v1/models/128x128

Since the status is:
```json
{
 "model_version_status": [
  {
   "version": "1",
   "state": "AVAILABLE",
   "status": {
    "error_code": "OK",
    "error_message": ""
   }
  }
 ]
}
```

it means that there are no issues with the local deployment

### Deploying Model Remotely

**Step 1**: Now for deploying remotely, we would have to set our root folder when deploying in render to be '/DLModel'. Then, we would also need a seperate models.config

``` bash
~: cd DLModel/img_classifier
~DLModel/img_classifier: mkdir remote_config
~DLModel/img_classifier: code models.config
```

**Step 2**: The content inside the model.config file will be similar to the one inside the one used for deploying it locally. However, we would have to modify the base path to match the root folder specified on render

Inside `models.config` we will add the following:
```json
model_config_list: { 
  config: {
    name: "31x31",
    base_path: "/img_classifier/31",
    model_platform: "tensorflow"
  },
  config: {
    name: "128x128",
    base_path: "/img_classifier/128",
    model_platform: "tensorflow"
  },
}
```


**Step 3**: Finally, we can create the docker container using tensorflow serving
```Dockerfile
FROM tensorflow/serving
COPY / /
ENV MODEL_CONF=/img_classifier/remote_config/models.config MODEL_BASE_PATH=/
EXPOSE 8500
EXPOSE 8501
RUN echo '#!/bin/bash \n\n\
tensorflow_model_server \
--rest_api_port=$PORT \
--model_config_file=${MODEL_CONF} \
"$@"' > /usr/bin/tf_serving_entrypoint.sh \
&& chmod +x /usr/bin/tf_serving_entrypoint.sh
```

**Step 4**: To test whether the server is running correctly, we would just have to check two urls:
- https://vegetablecnn.onrender.com/v1/models/31x31
- https://vegetablecnn.onrender.com/v1/models/128x128

Since the status is:
```json
{
 "model_version_status": [
  {
   "version": "1",
   "state": "AVAILABLE",
   "status": {
    "error_code": "OK",
    "error_message": ""
   }
  }
 ]
}
```

it means that there are no issues with the remote deployment